# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo
import datetime
from datetime import timedelta
import math
import pprint

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# query 1 

In [17]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax,zone, periode):

    dico={}
    query01={}
    query02={}
    center=[-1,-1]
    
    # points dans une zone Z
    if zone:
        
        center=[lon,lat]
        
        query01 = {
            "location.geo": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [ lon, lat]
                    },
                    "$minDistance": 0,
                    "$maxDistance": rad*1000 # mètres
                }
            }
        }

    # points à une période T
    if periode :
        query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}
    option= {'$and':[{'GPS_SPEED' :{"$ne": 0}},{'moving' : 1},{'GPS_DIR' : {"$ne": 0}}]} 
    query = {'$and': [query, option]}
        
    liste=list(collection.find(query))
    density=len(list(collection.find(query).distinct("asset_id")))
    dico['density']=density
    dico['points']=liste
    z = {}
    z['center'] = center
    z['radius'] = rad
    dico['zone'] = z
    
    return dico


# top zones

In [18]:
import math
import datetime

# Query 4
# Retournez le top k des périodes/ zones les plus denses 


# in meters
def distance(lat1, lat2, lon1, lon2):
    R = 6372800  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    d = 2 * R * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return d


def top_k_zones(k, rad,datemin,datemax):
    periode ={'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}
    # toutes les points de la base de données
    base = list(collection.find(periode))
    
    # tri selon la longitude puis la latitude
    base = sorted(base, key=lambda point:point['location']['geo']['coordinates'])
    print('sorting done\n')
    
    
    zones = []
    lon = 0
    lat = 0
    nextP = False
    
    for point in base:
        
        if distance(lat, point['location']['geo']['coordinates'][1],
                    lon, point['location']['geo']['coordinates'][0]) >= rad*1000*2:
            
            if len(zones) > 0:
                for z in zones:
                     
                    if distance(z['zone']['center'][1], point['location']['geo']['coordinates'][1],
                                z['zone']['center'][0], point['location']['geo']['coordinates'][0]) < rad*1000*2:
                        nextP = True
                        break
            if nextP:
                nextP = False
                continue
                
            lon = point['location']['geo']['coordinates'][0]
            lat = point['location']['geo']['coordinates'][1]
            
            d = density_list(lon, lat, rad, datemin, datemax, True, True)
            
            if d['density'] > 0:
                zones.append(d)
        
    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]


In [21]:
d1 = datetime.datetime(2020, 3, 2, 9, 0, 0)
d1 = datetime.datetime.timestamp(d1)
d2 = datetime.datetime(2020, 3, 2, 14, 0, 0)
d2 = datetime.datetime.timestamp(d2)
resultat=top_k_zones(5, 10,d1,d2)
resultat

sorting done



[{'density': 35,
  'points': [{'_id': ObjectId('5f32b1faf6d87d43fc5526be'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143893.0,
    'location': {'geo': {'coordinates': [2.07901, 48.7662], 'type': 'Point'}},
    'moving': 1,
    'GPS_DIR': 236.25,
    'GPS_SPEED': 4.65778},
   {'_id': ObjectId('5f32ba0a5d0d6da64f25801d'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143893.0,
    'location': {'geo': {'coordinates': [2.07901, 48.7662], 'type': 'Point'}},
    'moving': 1,
    'GPS_DIR': 236.25,
    'GPS_SPEED': 4.65778},
   {'_id': ObjectId('5f32b1faf6d87d43fc5526c1'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143913.0,
    'location': {'geo': {'coordinates': [2.07973, 48.7665], 'type': 'Point'}},
    'moving': 1,
    'GPS_DIR': 148.59,
    'GPS_SPEED': 1.65384},
   {'_id': ObjectId('5f32ba0a5d0d6da64f258020'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143913.0,
    'location': {'geo': {'coordinates': [2.07973, 48.7665], 'type':

In [46]:
def vitesse(dico):
    for zone in range(len(dico)):
        s=0
        for sum in range(len(dico[zone]['points'])):
            s+=dico[zone]['points'][sum]['GPS_SPEED']
        dico[zone]['vitesse']=s/len(dico[zone]['points'])
        
    return dico
resultat=top_k_zones(5, 10,d1,d2)
a=vitesse(resultat)
a

sorting done



[{'density': 35,
  'points': [{'_id': ObjectId('5f32b1faf6d87d43fc5526be'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143893.0,
    'location': {'geo': {'coordinates': [2.07901, 48.7662], 'type': 'Point'}},
    'moving': 1,
    'GPS_DIR': 236.25,
    'GPS_SPEED': 4.65778},
   {'_id': ObjectId('5f32ba0a5d0d6da64f25801d'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143893.0,
    'location': {'geo': {'coordinates': [2.07901, 48.7662], 'type': 'Point'}},
    'moving': 1,
    'GPS_DIR': 236.25,
    'GPS_SPEED': 4.65778},
   {'_id': ObjectId('5f32b1faf6d87d43fc5526c1'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143913.0,
    'location': {'geo': {'coordinates': [2.07973, 48.7665], 'type': 'Point'}},
    'moving': 1,
    'GPS_DIR': 148.59,
    'GPS_SPEED': 1.65384},
   {'_id': ObjectId('5f32ba0a5d0d6da64f258020'),
    'asset_id': '351622075249066',
    'recorded_at': 1583143913.0,
    'location': {'geo': {'coordinates': [2.07973, 48.7665], 'type':